# Project 6 - Pipleline 3 - Train

**Import libraries**

In [1]:
from os import chdir; chdir('../lib')
import encoding_module as enc
import database_module as db       
import wiki_module as wiki 
import pandas as pd

## Mine: Select document vectors for all pages from database 

This sill turn in to X (feature set)

In [48]:
SQLstr = """

select p.* from page_vec as p order by page_id asc
"""
pages_vector = db.select_statement(SQLstr, location="remote")

data_dict = {}
for index, row in pages_vector.iterrows():
    data_dict[row[0]]=row[1]
pages_vector_df = pd.DataFrame.from_dict(data_dict,orient='index')
pages_vector_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
17645570,0.039274,0.055023,-0.014005,-0.016520,0.022691,-0.018724,0.002756,0.005556,-0.005737,-0.009030,...,-0.010056,-0.014171,-0.013905,0.003224,0.005246,0.004800,0.032152,0.011491,0.022782,0.003275
37019651,0.101253,0.161288,-0.224323,0.112926,-0.060645,0.066798,0.023622,0.008715,-0.017338,0.008191,...,-0.001198,-0.016804,-0.015773,-0.009290,-0.000526,0.004315,-0.003473,0.003300,-0.009625,-0.005633
9109512,0.039232,0.039217,0.008973,-0.047728,-0.015612,0.017161,0.000355,-0.002134,-0.002816,-0.015128,...,0.031067,0.011480,-0.021530,-0.004341,0.004190,0.044702,0.011483,-0.013307,-0.031249,-0.063539


In [49]:
pages_vector_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2270 entries, 17645570 to 33660916
Columns: 500 entries, 0 to 499
dtypes: float64(500)
memory usage: 8.7 MB


## Mine: Select category ids corresponding to pages from database

This will turn in to y

In [56]:
SQLstr = """

select pc.page_id, pc.category_id from page_cate as pc, page_vec as pv where pc.page_id = pv.page_id and pc.category_id =
(select min(category_id) from page_cate pc1 where pc.page_id = pc1.page_id) 
order by pc.page_id asc
"""
target_df = db.select_statement(SQLstr, location="remote")


In [57]:
target_df.head(5)

,page_id,category_id
0,2443,18726608
1,2839,18726608
2,5180,691185
3,5931,34955640
4,7089,691015


In [58]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Data columns (total 2 columns):
page_id        2270 non-null int64
category_id    2270 non-null int64
dtypes: int64(2)
memory usage: 35.5 KB


In [60]:
del target_df["page_id"]

In [61]:
target_df.head()

,category_id
0,18726608
1,18726608
2,691185
3,34955640
4,691015


## Refine: Create a data dictionary with training and testing sets

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pages_vector_df,target_df, random_state=42)

data_dictionary = { 'X' : pages_vector_df, 
                    'y' : target_df,
                    'X_train': X_train,
                    'y_train': y_train,
                    'X_test' : X_test,
                    'y_test': y_test
                  }

In [64]:
data_dictionary.keys()

['X_test', 'X_train', 'y_train', 'y', 'X', 'y_test']

## Model: Fit, Score, and Tune a multi-label classification model

In [65]:
from sklearn.naive_bayes import GaussianNB

In [68]:
this_naive_calssifier = GaussianNB()
this_naive_calssifier.fit(data_dictionary['X_train'],data_dictionary['y_train'])

GaussianNB(priors=None)

In [69]:
train_score = this_naive_calssifier.score(data_dictionary['X_train'],
                                          data_dictionary['y_train'])

test_score = this_naive_calssifier.score(data_dictionary['X_test'],
                                         data_dictionary['y_test'])
print("train: {} test: {}".format(train_score, test_score))

train: 0.395417156287 test: 0.0845070422535


## Present: Pickle tuned model for later use

In [70]:
from sklearn.externals import joblib
joblib.dump(this_naive_calssifier, 'project6_pickled_model.pkl') 

['project6_pickled_model.pkl']

In [71]:
this_naive_calssifier = joblib.load('project6_pickled_model.pkl')

In [72]:
this_naive_calssifier.score(data_dictionary['X_train'],
                                          data_dictionary['y_train'])

0.39541715628672153